In [1]:
import random
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import networkx as nx
import json
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.model.general_recommender import *

c:\Users\CSE\anaconda3\envs\GL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

import numpy as np
import scipy.sparse as sp
import torch

from recbole.model.abstract_recommender import GeneralRecommender
from recbole.model.init import xavier_uniform_initialization
from recbole.model.loss import BPRLoss, EmbLoss
from recbole.utils import InputType


class CPALGC(GeneralRecommender):
    r"""LightGCN is a GCN-based recommender model.

    LightGCN includes only the most essential component in GCN — neighborhood aggregation — for
    collaborative filtering. Specifically, LightGCN learns user and item embeddings by linearly 
    propagating them on the user-item interaction graph, and uses the weighted sum of the embeddings
    learned at all layers as the final embedding.

    We implement the model following the original author with a pairwise training mode.
    """
    input_type = InputType.PAIRWISE

    def __init__(self, config, dataset, n_cri):
        super(CPALGC, self).__init__(config, dataset)

        # load dataset info
        self.interaction_matrix = dataset.inter_matrix(form='coo').astype(np.float32)
        self.cri_idx_shift = int((self.n_items)/n_cri) # should be replaced with variable later
        self.n_cri = n_cri

        # load parameters info
        self.latent_dim = config['embedding_size']  # int type:the embedding size of lightGCN
        self.n_layers = config['n_layers']  # int type:the layer num of lightGCN
        self.reg_weight = config['reg_weight']  # float32 type: the weight decay for l2 normalization

        # define layers and loss
        self.user_embedding = torch.nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.latent_dim)
        self.item_embedding = torch.nn.Embedding(num_embeddings=self.n_items, embedding_dim=self.latent_dim)
        self.cri_user_embedding = torch.nn.Embedding(num_embeddings=self.n_users, embedding_dim=self.latent_dim)
        self.cri_embedding_item = torch.zeros((self.n_items, self.latent_dim), device = config['device'])

        self.norm = PairNorm('PN', scale = 1)

        self.__init_criteria_weight()

    
        self.mf_loss = BPRLoss()
        self.reg_loss = EmbLoss()

        # storage variables for full sort evaluation acceleration
        self.restore_user_e = None
        self.restore_item_e = None

        # generate intermediate data
        self.norm_adj_matrix = self.get_norm_adj_mat_2().to(self.device)

        # parameters initialization
        self.apply(xavier_uniform_initialization)

    def __init_criteria_weight(self):

        with torch.no_grad():
            self.cri_emb_nograd = torch.zeros((self.n_cri, self.latent_dim))

        #nn.init.normal(self.cri_emb_nograd, std=0.1)
        torch.nn.init.xavier_uniform_(self.cri_emb_nograd)
        
        self.cri_emb_nograd[0] = torch.zeros(self.latent_dim)

        for i in range(self.n_items):
            if i // self.cri_idx_shift == 0:
                pass
            elif i // self.cri_idx_shift >= self.n_cri:
                self.cri_embedding_item[i] = self.cri_emb_nograd[self.n_cri-1]
            else:
                self.cri_embedding_item[i] = self.cri_emb_nograd[i // self.cri_idx_shift]
                 
            
    def get_norm_adj_mat(self):
        r"""Get the normalized interaction matrix of users and items.

        Construct the square matrix from the training data and normalize it
        using the laplace matrix.

        .. math::
            A_{hat} = D^{-0.5} \times A \times D^{-0.5}

        Returns:
            Sparse tensor of the normalized interaction matrix.
        """
        # build adj matrix
        A = sp.dok_matrix((self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32)
        inter_M = self.interaction_matrix
        inter_M_t = self.interaction_matrix.transpose()
        data_dict = dict(zip(zip(inter_M.row, inter_M.col + self.n_users), [1] * inter_M.nnz))
        data_dict.update(dict(zip(zip(inter_M_t.row + self.n_users, inter_M_t.col), [1] * inter_M_t.nnz)))
        A._update(data_dict)
        # norm adj matrix
        sumArr = (A > 0).sum(axis=1)
        # add epsilon to avoid divide by zero Warning
        diag = np.array(sumArr.flatten())[0] + 1e-7
        diag = np.power(diag, -0.5)
        D = sp.diags(diag)
        L = D * A * D
        # covert norm_adj matrix to tensor
        L = sp.coo_matrix(L)
        row = L.row
        col = L.col
        i = torch.LongTensor([row, col])
        data = torch.FloatTensor(L.data)
        SparseL = torch.sparse.FloatTensor(i, data, torch.Size(L.shape))
        return SparseL
            
    def get_norm_adj_mat_2(self):
        r"""Get the normalized interaction matrix of users and items.

        Construct the square matrix from the training data and normalize it
        using the laplace matrix.

        .. math::
            A_{hat} = D^{-0.5} \times A \times D^{-0.5}

        Returns:
            Sparse tensor of the normalized interaction matrix.
        """
        alpha = 1.5

        # build adj matrix
        A = sp.dok_matrix((self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32)
        inter_M = self.interaction_matrix
        inter_M_t = self.interaction_matrix.transpose()
        data_dict = dict(zip(zip(inter_M.row, inter_M.col + self.n_users), [alpha for _ in range(self.cri_idx_shift)] + [1] * inter_M.nnz))
        data_dict.update(dict(zip(zip(inter_M_t.row + self.n_users, inter_M_t.col), [alpha for _ in range(self.cri_idx_shift)] + [1] * inter_M.nnz)))
        A._update(data_dict)
        # norm adj matrix
        sumArr = (A > 0).sum(axis=1)
        # add epsilon to avoid divide by zero Warning
        diag = np.array(sumArr.flatten())[0] + 1e-7
        diag = np.power(diag, -0.5)
        D = sp.diags(diag)
        L = D * A * D
        # covert norm_adj matrix to tensor
        L = sp.coo_matrix(L)
        row = L.row
        col = L.col
        i = torch.LongTensor([row, col])
        data = torch.FloatTensor(L.data)
        SparseL = torch.sparse.FloatTensor(i, data, torch.Size(L.shape))
        return SparseL

    def get_ego_embeddings(self):
        r"""Get the embedding of users and items and combine to an embedding matrix.

        Returns:
            Tensor of the embedding matrix. Shape of [n_items+n_users, embedding_dim]
        """
        user_embeddings = self.user_embedding.weight
        item_embeddings = self.item_embedding.weight
        ego_embeddings = torch.cat([user_embeddings, item_embeddings], dim=0)

        cri_user_embeddings = self.cri_user_embedding.weight
        cri_item_embeddings = self.cri_embedding_item
        cri_ego_embeddings = torch.cat([cri_user_embeddings, cri_item_embeddings], dim=0)
        return ego_embeddings, cri_ego_embeddings 

    def forward(self):
        all_embeddings, cri_all_embeddings = self.get_ego_embeddings()
        all_embeddings = self.norm(all_embeddings)
        cri_all_embeddings = self.norm(cri_all_embeddings)
        embeddings_list = [all_embeddings]
        cri_embeddings_list = [cri_all_embeddings]

        for layer_idx in range(self.n_layers):
            all_embeddings = torch.sparse.mm(self.norm_adj_matrix, all_embeddings)
            all_embeddings = self.norm(all_embeddings)
            embeddings_list.append(all_embeddings)

        for layer_idx in range(self.n_layers):
            cri_all_embeddings = torch.sparse.mm(self.norm_adj_matrix, cri_all_embeddings)
            cri_all_embeddings = self.norm(cri_all_embeddings)
            cri_embeddings_list.append(cri_all_embeddings)

        lightgcn_all_embeddings = torch.stack(embeddings_list, dim=1)
        lightgcn_all_embeddings = torch.mean(lightgcn_all_embeddings, dim=1)

        lightgcn_cri_embeddings = torch.stack(cri_embeddings_list, dim=1)
        lightgcn_cri_embeddings = torch.mean(lightgcn_cri_embeddings, dim=1)

        lightgcn_all_embeddings = self.norm(lightgcn_all_embeddings)
        lightgcn_cri_embeddings = self.norm(lightgcn_cri_embeddings)

        lightgcn_all_embeddings = torch.cat([lightgcn_all_embeddings, lightgcn_cri_embeddings ], dim = 1)

        user_all_embeddings, item_all_embeddings = torch.split(lightgcn_all_embeddings, [self.n_users, self.n_items])

        return user_all_embeddings, item_all_embeddings


    def calculate_loss(self, interaction):
        # clear the storage variable when training
        if self.restore_user_e is not None or self.restore_item_e is not None:
            self.restore_user_e, self.restore_item_e = None, None

        user = interaction[self.USER_ID]
        pos_item = interaction[self.ITEM_ID]
        neg_item = interaction[self.NEG_ITEM_ID]

        user_all_embeddings, item_all_embeddings = self.forward()
        u_embeddings = user_all_embeddings[user]
        pos_embeddings = item_all_embeddings[pos_item]
        neg_embeddings = item_all_embeddings[neg_item]

        # calculate BPR Loss
        pos_scores = torch.mul(u_embeddings, pos_embeddings).sum(dim=1)
        neg_scores = torch.mul(u_embeddings, neg_embeddings).sum(dim=1)
        mf_loss = self.mf_loss(pos_scores, neg_scores)

        # calculate reg Loss
        u_ego_embeddings = self.user_embedding(user)
        pos_ego_embeddings = self.item_embedding(pos_item)
        neg_ego_embeddings = self.item_embedding(neg_item)

        reg_loss = self.reg_loss(u_ego_embeddings, pos_ego_embeddings, neg_ego_embeddings)
        loss = mf_loss + self.reg_weight * reg_loss

        return loss

    def predict(self, interaction):
        user = interaction[self.USER_ID]
        item = interaction[self.ITEM_ID]

        user_all_embeddings, item_all_embeddings = self.forward()

        u_embeddings = user_all_embeddings[user]
        i_embeddings = item_all_embeddings[item]
        scores = torch.mul(u_embeddings, i_embeddings).sum(dim=1)
        return scores

    def full_sort_predict(self, interaction):
        user = interaction[self.USER_ID]
        if self.restore_user_e is None or self.restore_item_e is None:
            self.restore_user_e, self.restore_item_e = self.forward()
        # get user embedding from storage variable
        u_embeddings = self.restore_user_e[user]
        # dot with all item embedding to accelerate
        scores = torch.matmul(u_embeddings, self.restore_item_e.transpose(0, 1))
        # We only consider overall interactions.
        scores[:,int(self.n_items/self.n_cri):] = -np.inf

        return scores.view(-1)



class PairNorm(torch.nn.Module):
    def __init__(self, mode='PN', scale=1):
        """
            mode:
              'None' : No normalization 
              'PN'   : Original version
              'PN-SI'  : Scale-Individually version
              'PN-SCS' : Scale-and-Center-Simultaneously version
           
            ('SCS'-mode is not in the paper but we found it works well in practice, 
              especially for GCN and GAT.)
            PairNorm is typically used after each graph convolution operation. 
        """
        assert mode in ['None', 'PN',  'PN-SI', 'PN-SCS']
        super(PairNorm, self).__init__()
        self.mode = mode
        self.scale = scale

        # Scale can be set based on origina data, and also the current feature lengths.
        # We leave the experiments to future. A good pool we used for choosing scale:
        # [0.1, 1, 10, 50, 100]
                
    def forward(self, x):
        if self.mode == 'None':
            return x
        
        col_mean = x.mean(dim=0)      
        if self.mode == 'PN':
            x = x - col_mean
            rownorm_mean = (1e-6 + x.pow(2).sum(dim=1).mean()).sqrt() 
            x = self.scale * x / rownorm_mean

        if self.mode == 'PN-SI':
            x = x - col_mean
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual

        if self.mode == 'PN-SCS':
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual - col_mean

        return x

In [4]:
dataset_name = 'TA5'
ncri_table = {'TA5':8, 'YM5':5, 'RB5':5, 'RA5':5, 'YP5':4}
n_cri = ncri_table[dataset_name]
epoch = 10
confg_dataset = {'benchmark_filename' : ['tr','val','ts']}
parameter_dict = {
    'benchmark_filename' : ['tr','val','ts'],
    'data_path': '',
    'seed': 2021,
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'user_inter_num_interval': "[0,inf)",
    'item_inter_num_interval': "[0,inf)",
    'load_col': {'inter': ['user_id', 'item_id']},
    'neg_sampling': None,
    'epochs': epoch,
    'metrics':['Precision', 'Recall', 'NDCG'],
    'topk':[5,10],
    'device': torch.device('cuda'),
    'embedding_size' : 64, 
    'n_layers' : 3,
    'learning_rate' : 1e-3,
    'reg_weight' : 1e-2,
}

config = Config(model='CPALGC', dataset=dataset_name, config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)


# Dataset preparation
datasets = create_dataset(config)

# dataset splitting 
train_data, valid_data, test_data = data_preparation(config, datasets)

# Model and learning,
# model loading and initialization
model = CPALGC(config, train_data.dataset, n_cri).to(config['device'])

# trainer loading and initialization
trainer = Trainer(config, model)

# model training 
best_valid_score, best_valid_result = trainer.fit(train_data)

# Evaluation
results = trainer.evaluate(test_data)
logger.info(results)

print(results)


01 Feb 14:21    INFO Build [ModelType.GENERAL] DataLoader for [train] with format [InputType.PAIRWISE]
Build [ModelType.GENERAL] DataLoader for [train] with format [InputType.PAIRWISE]
Build [ModelType.GENERAL] DataLoader for [train] with format [InputType.PAIRWISE]
01 Feb 14:21    INFO Evaluation Setting:
	Group by user_id
	Ordering: {'strategy': 'shuffle'}
	Splitting: {'strategy': 'by_ratio', 'ratios': [0.8, 0.1, 0.1]}
	Negative Sampling: {'strategy': 'by', 'distribution': 'uniform', 'by': 1}
Evaluation Setting:
	Group by user_id
	Ordering: {'strategy': 'shuffle'}
	Splitting: {'strategy': 'by_ratio', 'ratios': [0.8, 0.1, 0.1]}
	Negative Sampling: {'strategy': 'by', 'distribution': 'uniform', 'by': 1}
Evaluation Setting:
	Group by user_id
	Ordering: {'strategy': 'shuffle'}
	Splitting: {'strategy': 'by_ratio', 'ratios': [0.8, 0.1, 0.1]}
	Negative Sampling: {'strategy': 'by', 'distribution': 'uniform', 'by': 1}
01 Feb 14:21    INFO batch_size = [[2048]], shuffle = [True]

batch_size = [

{'precision@5': 0.0375, 'precision@10': 0.0249, 'recall@5': 0.0732, 'recall@10': 0.0968, 'ndcg@5': 0.0646, 'ndcg@10': 0.0735}
